# Machine Learning workflow en PyTorch

En este notebook vamos a cubrir un flujo de trabajo estándar de PyTorch (puede ser modificado según sea necesario, pero cubre los principales pasos).

<img src="https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/01_a_pytorch_workflow.png" width=900 alt="diagrama de flujo de trabajo de pytorch"/>

Por ahora, usaremos este flujo de trabajo para predecir una simple línea recta, pero los pasos pueden repetirse y modificarse según el problema en el que estés trabajando.

Específicamente, vamos a cubrir:

| **Tema** | **Contenido** |
| ----- | ----- |
| **1. Preparación de datos** | Los datos pueden ser casi cualquier cosa, pero para empezar vamos a crear una simple línea recta |
| **2. Construcción del modelo** | Aquí crearemos un modelo para aprender patrones en los datos, también elegiremos una **función de pérdida**, un **optimizador** y construiremos un **bucle de entrenamiento**. |
| **3. Ajuste del modelo a los datos (entrenamiento)** | Ya tenemos datos y un modelo, ahora dejemos que el modelo (intente) encontrar patrones en los datos de (**entrenamiento**). |
| **4. Hacer predicciones y evaluar el modelo (inferencia)** | Nuestro modelo encontró patrones en los datos, comparemos sus hallazgos con los datos (**de prueba**) reales. |
| **5. Guardar y cargar un modelo** | Podemos querer usar nuestro modelo en otro lugar o volver a él más tarde. |


Ahora importemos lo que necesitaremos para este módulo.

Vamos a obtener `torch`, `torch.nn` (`nn` significa red neuronal y este paquete contiene los componentes básicos para crear redes neuronales en PyTorch) y `matplotlib`.


In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt

torch.__version__

'2.6.0+cu124'

## 1. Datos (preparación y carga)

Los "datos" pueden ser casi cualquier cosa que se puedan imaginar. Una tabla de números (como una hoja de Excel), imágenes de cualquier tipo, videos, archivos de audio como canciones o podcasts, estructuras de proteínas, texto y más.

![el aprendizaje automático es un juego de dos partes: 1. convertir tus datos en un conjunto representativo de números y 2. construir o elegir un modelo para aprender la representación lo mejor posible](https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/01-machine-learning-a-game-of-two-parts.png)

El aprendizaje automático consta de dos partes:
1. Convertir tus datos, sean cuales sean, en números (una representación).
2. Elegir o construir un modelo para aprender la representación lo mejor posible.

Para este notebook crearemos nuestros datos y, luego, les tocará a ustedes trabajar con datasets ya creados.

Usaremos [regresión lineal](https://en.wikipedia.org/wiki/Linear_regression) para crear los datos con **parámetros** conocidos y luego usaremos PyTorch para ver si podemos construir un modelo para estimar estos parámetros usando [**descenso de gradiente**](https://en.wikipedia.org/wiki/Gradient_descent).

### Dataset y DataLoader

Las clases `Dataset` y `DataLoader` de PyTorch destacan por su capacidad para agilizar el preprocesamiento y la carga de datos.

La clase `Dataset` en PyTorch proporciona una interfaz para acceder a los datos. Permite definir cómo deben leerse, transformarse y accederse tus datos. La clase `DataLoader`, por otro lado, proporciona una forma eficiente de iterar sobre tu conjunto de datos en batches, lo cual es crucial para entrenar modelos.


Para crear un conjunto de datos personalizado, necesitamos definir una clase que herede de `torch.utils.data.Dataset`. Esta clase debe implementar tres métodos: `__init__`, `__len__`, y `__getitem__`.

* `__init__`: Inicializa el conjunto de datos con cualquier atributo necesario como rutas de archivos o pasos de preprocesamiento de datos.
* `__len__`: Devuelve el número total de muestras en tu conjunto de datos.
* `__getitem__`: Recupera una muestra del conjunto de datos dado un índice.

In [2]:
from torch.utils.data import Dataset, DataLoader, random_split

class LinearRegressionDataset(Dataset):
    """Custom Dataset for Linear Regression"""
    def __init__(self, start=0, end=1, step=0.02, weight=0.7, bias=0.3):
        """
        Initialize the dataset with given parameters

        Args:
            start (float): Starting value for X
            end (float): Ending value for X
            step (float): Step size between X values
            weight (float): Weight parameter for linear function (slope)
            bias (float): Bias parameter for linear function (intercept)
        """
        self.X = torch.arange(start, end, step).unsqueeze(dim=1)
        self.y = weight * self.X + bias

    def __len__(self):
        """Return the size of the dataset"""
        return len(self.X)

    def __getitem__(self, idx):
        """Get a sample from the dataset"""
        return self.X[idx], self.y[idx]


In [3]:
full_dataset = LinearRegressionDataset(start=0, end=1, step=0.001, weight=1, bias=0)

En el aprendizaje profundo, es fundamental separar nuestros datos en conjuntos de entrenamiento y prueba. Para manejar estos conjuntos de manera eficiente, utilizamos ```torch.utils.data.DataLoader```, que nos permite:

1. **Procesar los datos por batches**: Optimiza el uso de memoria y acelera el entrenamiento.
2. **Cargar los datos de manera eficiente**: Utiliza la API de ```torch.utils.data.Dataset``` para gestionar el acceso a los datos.
3. **Construir minibatches**: Permite definir el tamaño de batch según nuestras necesidades.

### Configuración de DataLoaders

En la práctica, necesitamos dos DataLoaders diferentes:

1. **DataLoader de Entrenamiento**:
   * Se configura con ```shuffle=True```
   * Aleatoriza las muestras en cada epoch
   * Ayuda a prevenir el sobreajuste (overfitting)
   * Asegura que la red vea combinaciones diferentes de datos en cada epoch

2. **DataLoader de Validación**:
   * Se configura con ```shuffle=False```
   * Mantiene un orden consistente de los datos

Vamos a crear los dos dataloaders:

In [4]:
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

# Separar el dataset
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Crear dataloaders
batch_size = 8

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,  # No es necesario shufflear el test
    num_workers=0
)

Ahora vamos a querer aprender los parámetros teniendo a `X` (**features**) e `y` (**labels**).

## 2. Construir el modelo

Ahora que tenemos algunos datos, vamos a construir un modelo para intentar aprender los parámetros desconocidos.

En este caso replicaremos un modelo estándar de regresión lineal usando PyTorch puro. (Acá es donde pueden armar la arquitectura que quieran, convolucionales, RNNs)

In [5]:
# Crear una clase de modelo de Regresión Lineal
class LinearRegressionModel(nn.Module): # <- casi todo en PyTorch es un nn.Module (piensa en esto como bloques de lego de redes neuronales)
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,  # <- comenzar con pesos aleatorios (esto se ajustará mientras el modelo aprende)
                                                dtype=torch.float),  # <- PyTorch prefiere float32 por defecto
                                    requires_grad=True)  # <- ¿podemos actualizar este valor con descenso de gradiente?

        self.bias = nn.Parameter(torch.randn(1, # <- comenzar con sesgo aleatorio (esto se ajustará mientras el modelo aprende)
                                             dtype=torch.float), # <- PyTorch prefiere float32 por defecto
                                 requires_grad=True) # <- ¿podemos actualizar este valor con descenso de gradiente?


    # Forward define la pasada forward en el modelo
    def forward(self, x: torch.Tensor) -> torch.Tensor: # <- "x" son los datos de entrada (
        return self.weights * x + self.bias # <- esta es la fórmula de regresión lineal (y = m*x + b)


> **Más info:** Usaremos clases de Python para crear diferentes partes para construir redes neuronales. Si no están familiarizados con la notación de clases en Python, recomiendo leer varias veces la [guía de Programación Orientada a Objetos en Python 3 de Real Python](https://realpython.com/python3-object-oriented-programming/).


### Elementos esenciales para construir modelos en PyTorch

PyTorch tiene cuatro (más o menos) módulos esenciales que puedes usar para crear casi cualquier tipo de red neuronal.

Estos son [`torch.nn`](https://pytorch.org/docs/stable/nn.html), [`torch.optim`](https://pytorch.org/docs/stable/optim.html), [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) y [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html).

| Módulo PyTorch | ¿Qué hace? |
| ----- | ----- |
| [`torch.nn`](https://pytorch.org/docs/stable/nn.html) | Contiene todos los bloques de construcción para grafos computacionales (esencialmente una serie de cálculos ejecutados de una manera particular). |
| [`torch.nn.Parameter`](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#parameter) | Almacena tensores que pueden ser usados con `nn.Module`. Si `requires_grad=True`, los gradientes (usados para actualizar parámetros del modelo mediante [**descenso de gradiente**](https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html)) se calculan automáticamente, esto se conoce frecuentemente como "autograd". |
| [`torch.nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) | La clase base para todos los módulos de redes neuronales, todos los bloques de construcción para redes neuronales son subclases. Si estás construyendo una red neuronal en PyTorch, tus modelos deberían heredar de `nn.Module`. Requiere que se implemente un método `forward()`. |
| [`torch.optim`](https://pytorch.org/docs/stable/optim.html) | Contiene varios algoritmos de optimización (estos le dicen a los parámetros del modelo almacenados en `nn.Parameter` cómo cambiar mejor para mejorar el descenso de gradiente y a su vez reducir la pérdida). |
| `def forward()` | Todas las subclases de `nn.Module` requieren un método `forward()`, esto define el cálculo que se realizará en los datos pasados al `nn.Module` particular (por ejemplo, la fórmula de regresión lineal anterior). |

Si lo anterior suena complejo, piénsalo así, casi todo en una red neuronal de PyTorch viene de `torch.nn`,
* `nn.Module` contiene los bloques de construcción más grandes (capas)
* `nn.Parameter` contiene los parámetros más pequeños como pesos y sesgos
* `forward()` le dice a los bloques más grandes cómo hacer cálculos en las entradas (tensores llenos de datos) dentro de los `nn.Module`(s)
* `torch.optim` contiene métodos de optimización sobre cómo mejorar los parámetros dentro de `nn.Parameter` para representar mejor los datos de entrada


> **Más info:** [Hoja de Trucos de PyTorch](https://pytorch.org/tutorials/beginner/ptcheat.html).


### Revisando el contenido de un modelo PyTorch

Vamos a crear una instancia del modelo con la clase que creamos y revisar sus parámetros usando [`.parameters()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters).

In [6]:
# Establecer semilla manual ya que nn.Parameter se inicializa aleatoriamente
torch.manual_seed(42)

# Crear una instancia del modelo (esto es una subclase de nn.Module que contiene nn.Parameter(s))
model_0 = LinearRegressionModel()

# Revisar los nn.Parameter(s) dentro de la subclase nn.Module que creamos
list(model_0.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

También podemos obtener el estado (lo que contiene el modelo) del modelo usando [`.state_dict()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.state_dict).


In [7]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

### Haciendo predicciones usando `torch.inference_mode()`

Podemos pasarle los datos de prueba `X_test` para ver qué tan cerca predice `y_test`.

Cuando pasamos datos a nuestro modelo, estos pasarán por el método `forward()` del modelo y producirán un resultado usando lo que definimos.


In [8]:
predictions = []
with torch.inference_mode():  # o torch.no_grad()
    for X_batch, y_batch in test_loader:
        batch_preds = model_0(X_batch)
        predictions.append(batch_preds)

    y_preds = torch.cat(predictions)

print(y_preds)

# with torch.no_grad():
#   y_preds = model_0(X_test)

tensor([[0.4244],
        [0.1601],
        [0.3211],
        [0.1655],
        [0.2312],
        [0.3854],
        [0.4557],
        [0.3823],
        [0.2079],
        [0.3847],
        [0.1864],
        [0.3352],
        [0.4110],
        [0.3591],
        [0.3076],
        [0.1413],
        [0.2493],
        [0.4318],
        [0.2066],
        [0.2652],
        [0.1379],
        [0.2009],
        [0.3709],
        [0.3860],
        [0.3716],
        [0.2568],
        [0.2271],
        [0.1288],
        [0.2288],
        [0.3248],
        [0.4493],
        [0.3911],
        [0.1487],
        [0.4301],
        [0.4574],
        [0.1430],
        [0.2753],
        [0.3557],
        [0.1898],
        [0.2615],
        [0.2012],
        [0.1985],
        [0.3160],
        [0.3302],
        [0.1648],
        [0.1807],
        [0.4362],
        [0.2982],
        [0.2443],
        [0.3093],
        [0.2756],
        [0.4477],
        [0.1376],
        [0.1446],
        [0.2504],
        [0

Como sugiere el nombre, `torch.inference_mode()` se usa cuando se utiliza un modelo para inferencia (hacer predicciones).

`torch.inference_mode()` desactiva varias cosas (como el seguimiento de gradientes, que es necesario para el entrenamiento pero no para la inferencia) para hacer los **pases hacia adelante** (datos pasando por el método `forward()`) más rápidos.

> **Nota:** En código más antiguo de PyTorch, también pueden encontrar `torch.no_grad()` siendo usado para inferencia. Aunque `torch.inference_mode()` y `torch.no_grad()` hacen cosas similares, `torch.inference_mode()` es más nuevo, potencialmente más rápido y preferido. Mira este [Tweet de PyTorch](https://twitter.com/PyTorch/status/1437838231505096708?s=20) para más información.


In [9]:
# Chequear predicciones
print(f"Número de muestras de prueba: {test_size}")
print(f"Número de predicciones realizadas: {len(y_preds)}")

Número de muestras de prueba: 200
Número de predicciones realizadas: 200


Observemos cómo hay un valor de predicción por cada muestra de prueba.

Esto es debido al tipo de datos que estamos usando. Para nuestra línea recta, un valor de `X` se corresponde con un valor de `y`.

Sin embargo, los modelos de aprendizaje automático son muy flexibles. Podríamos tener 100 valores de `X` correspondiendo a uno, dos, tres o 10 valores de `y`. Todo depende de lo que estemos trabajando.

In [10]:
y_test = torch.cat([batch[1] for batch in test_loader])

In [11]:
y_test - y_preds

tensor([[ 4.5358e-01],
        [-6.7122e-02],
        [ 2.4994e-01],
        [-5.6509e-02],
        [ 7.2837e-02],
        [ 3.7663e-01],
        [ 5.1526e-01],
        [ 3.7066e-01],
        [ 2.7068e-02],
        [ 3.7531e-01],
        [-1.5383e-02],
        [ 2.7780e-01],
        [ 4.2704e-01],
        [ 3.2489e-01],
        [ 2.2341e-01],
        [-1.0427e-01],
        [ 1.0866e-01],
        [ 4.6817e-01],
        [ 2.4415e-02],
        [ 1.3983e-01],
        [-1.1090e-01],
        [ 1.3139e-02],
        [ 3.4811e-01],
        [ 3.7796e-01],
        [ 3.4944e-01],
        [ 1.2325e-01],
        [ 6.4877e-02],
        [-1.2881e-01],
        [ 6.8194e-02],
        [ 2.5724e-01],
        [ 5.0266e-01],
        [ 3.8791e-01],
        [-8.9674e-02],
        [ 4.6485e-01],
        [ 5.1858e-01],
        [-1.0095e-01],
        [ 1.5973e-01],
        [ 3.1826e-01],
        [-8.7504e-03],
        [ 1.3253e-01],
        [ 1.3802e-02],
        [ 8.4957e-03],
        [ 2.3999e-01],
        [ 2

Las predicciones están bastante lejos de la realidad. Esto tiene sentido cuando recordamos que nuestro modelo está usando solo valores de parámetros aleatorios para hacer predicciones.


## 3. Entrenar el modelo

Ahora mismo nuestro modelo está haciendo predicciones usando parámetros aleatorios para hacer cálculos, básicamente está adivinando (al azar).

Para arreglar esto, podemos actualizar sus parámetros internos, los valores de `weights` (pesos) y `bias` (sesgo) que establecimos aleatoriamente usando `nn.Parameter()` y `torch.randn()` para que sean algo que represente mejor los datos.


### Creando una función de pérdida y un optimizador en PyTorch

Necesitamos agregar una **función de pérdida** y un **optimizador**.

Sus roles son:

| Función | ¿Qué hace? | ¿Dónde se encuentra en PyTorch? | Valores comunes |
| ----- | ----- | ----- | ----- |
| **Función de pérdida** | Mide qué tan equivocadas están las predicciones de tu modelo (ej. `y_preds`) comparadas con las etiquetas verdaderas (ej. `y_test`). Mientras más bajo, mejor. | PyTorch tiene muchas funciones de pérdida incorporadas en [`torch.nn`](https://pytorch.org/docs/stable/nn.html#loss-functions). | Error absoluto medio (MAE) para problemas de regresión ([`torch.nn.L1Loss()`](https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html)). Entropía cruzada binaria para problemas de clasificación binaria ([`torch.nn.BCELoss()`](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html)). |
| **Optimizador** | Le dice a nuestro modelo cómo actualizar sus parámetros internos para reducir mejor la pérdida. | Pueden encontrar varias implementaciones de funciones de optimización en [`torch.optim`](https://pytorch.org/docs/stable/optim.html). | Descenso de gradiente estocástico ([`torch.optim.SGD()`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)). Optimizador Adam ([`torch.optim.Adam()`](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam)). |


Para nuestro problema, ya que estamos prediciendo un número, usemos MAE (que está bajo `torch.nn.L1Loss()`) en PyTorch como nuestra función de pérdida.

Y usaremos SGD, `torch.optim.SGD(params, lr)` donde:

* `params` son los parámetros objetivo del modelo que nos gustaría optimizar (ej. los valores de `weights` y `bias` que establecimos aleatoriamente antes).
* `lr` es la **tasa de aprendizaje** a la que nos gustaría que el optimizador actualice los parámetros, más alta significa que el optimizador intentará actualizaciones más grandes (estas pueden ser a veces demasiado grandes y el optimizador fallará), más baja significa que el optimizador intentará actualizaciones más pequeñas (estas pueden ser a veces demasiado pequeñas y el optimizador tardará demasiado en encontrar los valores ideales). La tasa de aprendizaje se considera un **hiperparámetro** (porque es establecida por un ingeniero de aprendizaje automático). Los valores iniciales comunes para la tasa de aprendizaje son `0.01`, `0.001`, `0.0001`, sin embargo, estos también pueden ajustarse con el tiempo (esto se llama [programación de tasa de aprendizaje](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)).

In [12]:
# Crear la función de pérdida
loss_fn = nn.L1Loss() # La pérdida MAE es lo mismo que L1Loss

# Crear el optimizador
optimizer = torch.optim.SGD(
    params=model_0.parameters(), # parámetros del modelo objetivo para optimizar
    lr=0.01 # tasa de aprendizaje
            # (cuánto debe cambiar el optimizador los parámetros en cada paso, mayor=más (menos estable), menor=menos (podría tomar mucho tiempo))
)

### Creando un bucle de optimización en PyTorch

Ahora que tenemos una función de pérdida y un optimizador, es hora de crear un **bucle de entrenamiento** (y un **bucle de prueba**).

El bucle de entrenamiento implica que el modelo recorra los datos de entrenamiento y aprenda las relaciones entre las `features` (características) y las `labels` (etiquetas).

El bucle de prueba implica recorrer los datos de prueba y evaluar qué tan buenos son los patrones que el modelo aprendió en los datos de entrenamiento (el modelo nunca ve los datos de prueba durante el entrenamiento).

### Bucle de entrenamiento en PyTorch

Para el bucle de entrenamiento, construiremos los siguientes pasos:

| Número | Nombre del paso | ¿Qué hace? | Ejemplo de código |
| ----- | ----- | ----- | ----- |
| 1 | forward pass | El modelo recorre todos los datos de entrenamiento una vez, realizando los cálculos de su función `forward()`. | `model(x_train)` |
| 2 | Calcular la pérdida | Las salidas del modelo (predicciones) se comparan con la verdad fundamental y se evalúan para ver qué tan equivocadas están. | `loss = loss_fn(y_pred, y_train)` |
| 3 | Poner gradientes a cero | Los gradientes del optimizador se ponen a cero (se acumulan por defecto) para que puedan ser recalculados para el paso de entrenamiento específico. | `optimizer.zero_grad()` |
| 4 | Realizar retropropagación en la pérdida | Calcula el gradiente de la pérdida con respecto a cada parámetro del modelo que se actualizará (cada parámetro con `requires_grad=True`). Esto se conoce como **retropropagación** o **backpropagation**, de ahí "backwards". | `loss.backward()` |
| 5 | Actualizar el optimizador (**descenso de gradiente**) | Actualiza los parámetros con `requires_grad=True` con respecto a los gradientes de pérdida para mejorarlos. | `optimizer.step()` |

![bucle de entrenamiento pytorch anotado](https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/01-pytorch-training-loop-annotated.png)

> Y sobre el orden de las cosas, el anterior es un buen orden por defecto, pero puedes ver órdenes ligeramente diferentes. Algunas reglas generales:
> * Calcular la pérdida (`loss = ...`) *antes* de realizar la retropropagación en ella (`loss.backward()`).
> * Poner los gradientes a cero (`optimizer.zero_grad()`) *antes* de calcular los gradientes de la pérdida con respecto a cada parámetro del modelo (`loss.backward()`).
> * Dar un paso en el optimizador (`optimizer.step()`) *después* de realizar la retropropagación en la pérdida (`loss.backward()`).

### Bucle de testing en PyTorch

En cuanto al bucle de testing (evaluación de nuestro modelo), los pasos típicos incluyen:

| Número | Nombre del paso | ¿Qué hace? | Ejemplo de código |
| ----- | ----- | ----- | ----- |
| 1 | forward pass | El modelo recorre todos los datos de prueba una vez, realizando los cálculos de su función `forward()`. | `model(x_test)` |
| 2 | Calcular la pérdida | Las salidas del modelo (predicciones) se comparan con la verdad y se evalúan para ver qué tan equivocadas están. | `loss = loss_fn(y_pred, y_test)` |
| 3 | Calcular métricas de evaluación (opcional) | Junto con el valor de pérdida, puedes querer calcular otras métricas de evaluación como la precisión en el conjunto de prueba. | Funciones personalizadas |

Este bucle no contiene la realización de retropropagación (`loss.backward()`) ni el paso del optimizador (`optimizer.step()`), esto es porque ningún parámetro en el modelo está siendo cambiado durante la prueba. Para las pruebas, solo nos interesa la salida del pase hacia adelante a través del modelo.

![bucle de prueba pytorch anotado](https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/01-pytorch-testing-loop-annotated.png)

In [13]:
torch.manual_seed(42)

epochs = 100

# Empty lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ### Training
    model_0.train()
    train_loss = 0

    # Loop through training batches
    for X_batch, y_batch in train_loader:
        # 1. Forward pass
        y_pred = model_0(X_batch)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y_batch)

        # 3. Zero gradients
        optimizer.zero_grad()

        # 4. Backpropagation
        loss.backward()

        # 5. Update parameters
        optimizer.step()

        # Accumulate batch loss
        train_loss += loss.item()

    # Calculate average training loss for the epoch
    train_loss = train_loss / len(train_loader)

    ### Testing
    model_0.eval()
    test_loss = 0

    with torch.inference_mode():
        # Loop through test batches
        for X_batch, y_batch in test_loader:
            # 1. Forward pass on test data
            test_pred = model_0(X_batch)

            # 2. Calculate test loss
            batch_test_loss = loss_fn(test_pred, y_batch)

            # Accumulate batch test loss
            test_loss += batch_test_loss.item()

        # Calculate average test loss for the epoch
        test_loss = test_loss / len(test_loader)

    # Print progress every 10 epochs
    if epoch % 10 == 0:
        epoch_count.append(epoch)
        train_loss_values.append(train_loss)
        test_loss_values.append(test_loss)
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.5f} | Test Loss: {test_loss:.5f}")

Epoch: 0 | Train Loss: 0.13338 | Test Loss: 0.09890
Epoch: 10 | Train Loss: 0.00596 | Test Loss: 0.00596
Epoch: 20 | Train Loss: 0.00619 | Test Loss: 0.01175
Epoch: 30 | Train Loss: 0.00629 | Test Loss: 0.00551
Epoch: 40 | Train Loss: 0.00616 | Test Loss: 0.00782
Epoch: 50 | Train Loss: 0.00616 | Test Loss: 0.00370
Epoch: 60 | Train Loss: 0.00628 | Test Loss: 0.00098
Epoch: 70 | Train Loss: 0.00626 | Test Loss: 0.00815
Epoch: 80 | Train Loss: 0.00629 | Test Loss: 0.00221
Epoch: 90 | Train Loss: 0.00615 | Test Loss: 0.00631



Vamos a inspeccionar el [`.state_dict()`](https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html) de nuestro modelo para ver qué tan cerca llega nuestro modelo a los valores originales que establecimos para los pesos y el sesgo.


In [14]:
# Revisar los parámetros del modelo
print("\nParámetros del modelo:")
print(model_0.state_dict())

print("\nValores originales:")
print(f"weights: 1.0, bias: 0.0")


Parámetros del modelo:
OrderedDict([('weights', tensor([1.0059])), ('bias', tensor([0.0013]))])

Valores originales:
weights: 1.0, bias: 0.0


## 4. Haciendo predicciones con un modelo PyTorch entrenado (inferencia)

Una vez que entrenamos un modelo, queremos hacer predicciones con él.

Hay tres cosas para recordar cuando se hacen predicciones (también llamado realizar inferencia) con un modelo PyTorch:

1. Establecer el modelo en modo evaluación (`model.eval()`).
2. Hacer las predicciones usando `with torch.inference_mode(): ...`.
3. Todas las predicciones deben hacerse con objetos en el mismo dispositivo (por ejemplo, datos y modelo solo en GPU o datos y modelo solo en CPU).

Los primeros dos elementos aseguran que todos los cálculos y configuraciones útiles que PyTorch usa detrás de escenas durante el entrenamiento, pero que no son necesarios para la inferencia, estén desactivados (esto resulta en un cómputo más rápido). Y el tercero asegura que no te encontrarás con errores entre dispositivos.

In [15]:
# 1. Establecer el modelo en modo evaluación
model_0.eval()

# 2. Calcular predicciones
with torch.inference_mode():
    predictions = []
    true_values = []

    for X_batch, y_batch in test_loader:
        # Opcional: Mover los datos al dispositivo necesario
        # X_batch = X_batch.to(device)
        # y_batch = y_batch.to(device)

        # Inferencia
        batch_preds = model_0(X_batch)

        # Guardar predicciones y real
        predictions.append(batch_preds)
        true_values.append(y_batch)

    # Combinar las predicciones de todos los batches
    y_preds = torch.cat(predictions)
    y_test = torch.cat(true_values)

## 5. Guardando y cargando un modelo PyTorch

Si entrenamos un modelo PyTorch, es probable que queramos guardarlo y exportarlo a algún lugar.

Para guardar y cargar modelos en PyTorch, hay tres métodos principales que deberiamos conocer (todo lo siguiente viene de la [guía de guardado y carga de modelos de PyTorch](https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-model-for-inference)):

| Método PyTorch | ¿Qué hace? |
| ----- | ----- |
| [`torch.save`](https://pytorch.org/docs/stable/torch.html?highlight=save#torch.save) | Guarda un objeto serializado en el disco usando la utilidad [`pickle`](https://docs.python.org/3/library/pickle.html) de Python. Los modelos, tensores y varios otros objetos de Python como diccionarios pueden guardarse usando `torch.save`. |
| [`torch.load`](https://pytorch.org/docs/stable/torch.html?highlight=torch%20load#torch.load) | Usa las características de despickling de `pickle` para deserializar y cargar archivos de objetos Python guardados (como modelos, tensores o diccionarios) en la memoria. También puedes establecer a qué dispositivo cargar el objeto (CPU, GPU, etc.). |
| [`torch.nn.Module.load_state_dict`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=load_state_dict#torch.nn.Module.load_state_dict) | Carga el diccionario de parámetros de un modelo (`model.state_dict()`) usando un objeto `state_dict()` guardado. |


### Guardando el `state_dict()` de un modelo PyTorch

La [forma recomendada](https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-model-for-inference) para guardar y cargar un modelo para inferencia (hacer predicciones) es guardando y cargando el `state_dict()` del modelo.

Veamos cómo podemos hacer esto en algunos pasos:

1. Crearemos un directorio llamado `models` para guardar modelos usando el módulo `pathlib` de Python.
2. Crearemos una ruta de archivo para guardar el modelo.
3. Llamaremos a `torch.save(obj, f)` donde `obj` es el `state_dict()` del modelo objetivo y `f` es el nombre del archivo donde guardar el modelo.

> **Nota:** Es una convención común que los modelos u objetos guardados de PyTorch terminen con `.pt` o `.pth`, como `saved_model_01.pth`.

In [16]:
# 1. Crear un directorio para guardar modelos
from pathlib import Path
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Crear nombre de archivo para guardar el modelo
MODEL_NAME = "modelo_regresion_01.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Guardar el state_dict del modelo
print(f"Guardando modelo en: {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(),
          f=MODEL_SAVE_PATH)

Guardando modelo en: models/modelo_regresion_01.pth


In [17]:
# Chequear que esta guardado
!ls -l models/modelo_regresion_01.pth

-rw-r--r-- 1 root root 1568 Jun  5 21:57 models/modelo_regresion_01.pth


### Cargando el `state_dict()` de un modelo PyTorch guardado

Tenemos un `state_dict()` guardado en `models/models/modelo_regresion_01.pth`, podemos cargarlo usando `torch.nn.Module.load_state_dict(torch.load(f))` donde `f` es la ruta del archivo de nuestro `state_dict()` guardado.

¿Por qué llamar a `torch.load()` dentro de `torch.nn.Module.load_state_dict()`?

Porque solo guardamos el `state_dict()` del modelo, que es un diccionario de parámetros aprendidos y no el modelo *completo*, primero tenemos que cargar el `state_dict()` con `torch.load()` y luego pasar ese `state_dict()` a una nueva instancia de nuestro modelo (que es una subclase de `nn.Module`).

Vamos a probarlo creando otra instancia de `LinearRegressionModel()`, que es una subclase de `torch.nn.Module` y por lo tanto tendrá el método incorporado `load_state_dict()`.

In [18]:
# Crear una nueva instancia de nuestro modelo (esto se iniciará con pesos aleatorios)
loaded_model_0 = LinearRegressionModel()

# Cargar el state_dict de nuestro modelo guardado (esto actualizará la nueva instancia de nuestro modelo con los pesos entrenados)
loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [19]:
# 1. Poner el modelo cargado en modo evaluación
loaded_model_0.eval()

# 2. Usar el administrador de contexto de modo inferencia para hacer predicciones
with torch.inference_mode():
    loaded_model_preds = []

    for X_batch, y_batch in test_loader:

      loaded_model_preds_batch = loaded_model_0(X_batch) # realizar un pase hacia adelante en los datos de prueba con el modelo cargado
      loaded_model_preds.append(loaded_model_preds_batch)

loaded_model_preds = torch.cat(loaded_model_preds)

Ahora veamos si son las mismas que las predicciones anteriores.


In [20]:
# Comparar las predicciones del modelo anterior con las predicciones del modelo cargado (deberían ser las mismas)
print(f"Predicciones previas:\n{y_preds}\n")
print(f"Predicciones del modelo cargado:\n{loaded_model_preds}\n")
print(f"¿Son iguales?: {torch.all(torch.eq(y_preds, loaded_model_preds))}")

Predicciones previas:
tensor([[0.8845],
        [0.0949],
        [0.5757],
        [0.1109],
        [0.3071],
        [0.7678],
        [0.9780],
        [0.7587],
        [0.2377],
        [0.7658],
        [0.1733],
        [0.6179],
        [0.8442],
        [0.6893],
        [0.5354],
        [0.0385],
        [0.3614],
        [0.9066],
        [0.2337],
        [0.4087],
        [0.0285],
        [0.2166],
        [0.7245],
        [0.7698],
        [0.7265],
        [0.3835],
        [0.2950],
        [0.0013],
        [0.3001],
        [0.5867],
        [0.9589],
        [0.7849],
        [0.0607],
        [0.9016],
        [0.9830],
        [0.0436],
        [0.4389],
        [0.6793],
        [0.1834],
        [0.3976],
        [0.2176],
        [0.2095],
        [0.5606],
        [0.6028],
        [0.1089],
        [0.1562],
        [0.9197],
        [0.5073],
        [0.3463],
        [0.5405],
        [0.4399],
        [0.9539],
        [0.0275],
        [0.0486],
      

## 6. Tarea